In [1]:
using Dates
using MarketData
using StatsBase
using Plots
using DataFrames
using Statistics
using MarketTechnicals
using Lathe
using TimeSeries
using MLJ,MLJBase

In [14]:

include("HyperParameterTuning.jl")

In [3]:
btc = yahoo("BTC-USD", YahooOpt(period1 = DateTime(2021, 1, 1), period2 = DateTime(2021, 12, 1)))["Close"]
btc_df = DataFrame(btc[55:end])
btc_df[!, "EMA 55"] = ema(values(btc), 55)[1:end]
btc_df[!, "EMA 15"] = ema(values(btc), 15)[55 - 15 + 1: end]
btc_df[!, :Cross] .= 0
btc_df[!, :Cross] = ifelse.(btc_df[:, "EMA 55"] .< btc_df[:, "EMA 15"], 1, 0)
btc_ts = TimeArray(btc_df, timestamp = :timestamp)
targetPositions = string.(btc_ts.Cross) # labels
data = btc_ts[:Close, Symbol("EMA 55"), Symbol("EMA 15")]
times = TimeArray(DataFrame(startTime = timestamp(btc_ts), endTime = append!(timestamp(btc_ts)[2:end], [timestamp(btc_ts)[end] + (timestamp(btc_ts)[2]-timestamp(btc_ts)[1])])), timestamp = :startTime)
#train_index, test_index = Lathe.preprocess.UniformSplit(collect(1:length(btc_ts)))

281×1 TimeArray{Date, 1, Date, Vector{Date}} 2021-02-24 to 2021-12-01
│            │ endTime            │
├────────────┼────────────────────┤
│ 2021-02-24 │ Date("2021-02-25") │
│ 2021-02-25 │ Date("2021-02-26") │
│ 2021-02-26 │ Date("2021-02-27") │
│ 2021-02-27 │ Date("2021-02-28") │
│ 2021-02-28 │ Date("2021-03-01") │
│ 2021-03-01 │ Date("2021-03-02") │
│ 2021-03-02 │ Date("2021-03-03") │
│ 2021-03-03 │ Date("2021-03-04") │
│ 2021-03-04 │ Date("2021-03-05") │
│ 2021-03-05 │ Date("2021-03-06") │
│ 2021-03-06 │ Date("2021-03-07") │
│ 2021-03-07 │ Date("2021-03-08") │
   ⋮
│ 2021-11-21 │ Date("2021-11-22") │
│ 2021-11-22 │ Date("2021-11-23") │
│ 2021-11-23 │ Date("2021-11-24") │
│ 2021-11-24 │ Date("2021-11-25") │
│ 2021-11-25 │ Date("2021-11-26") │
│ 2021-11-26 │ Date("2021-11-27") │
│ 2021-11-27 │ Date("2021-11-28") │
│ 2021-11-28 │ Date("2021-11-29") │
│ 2021-11-29 │ Date("2021-11-30") │
│ 2021-11-30 │ Date("2021-12-01") │
│ 2021-12-01 │ Date("2021-12-02") │

In [4]:
purgedKFold = PurgedKFold(5, times, 0.05)

PurgedKFold(
    nSplits = 5,
    times = 281×1 TimeArray{Date, 1, Date, Vector{Date}} 2021-02-24 to 2021-12-01,
    percentEmbargo = 0.05) @247

In [5]:
@load DecisionTreeClassifier
tree = DecisionTreeClassifier()

DecisionTreeClassifier(
    max_depth = -1,
    min_samples_leaf = 1,
    min_samples_split = 2,
    min_purity_increase = 0.0,
    n_subfeatures = 0,
    post_prune = false,
    merge_purity_threshold = 1.0,
    pdf_smoothing = 0.0,
    display_depth = 5,
    rng = Random._GLOBAL_RNG()) @000

In [7]:
r = range(tree, :min_purity_increase, lower=0.001, upper=1.0, scale=:log);
self_tuning_tree = TunedModel(model=tree,
							  resampling=purgedKFold,
							  tuning=Grid(resolution=10),
							  range=r,
							  measure=nothing);

┌ Info: No measure specified. Setting measure=CrossEntropy{Float64} @141. 
└ @ MLJTuning C:\Users\Ahmad\.julia\packages\MLJTuning\6MZ7C\src\tuned_models.jl:222


In [8]:
self_tuning_tree 

ProbabilisticTunedModel(
    model = DecisionTreeClassifier(
            max_depth = -1,
            min_samples_leaf = 1,
            min_samples_split = 2,
            min_purity_increase = 0.0,
            n_subfeatures = 0,
            post_prune = false,
            merge_purity_threshold = 1.0,
            pdf_smoothing = 0.0,
            display_depth = 5,
            rng = Random._GLOBAL_RNG()),
    tuning = Grid(
            goal = nothing,
            resolution = 10,
            shuffle = true,
            rng = Random._GLOBAL_RNG()),
    resampling = PurgedKFold(
            nSplits = 5,
            times = 281×1 TimeArray{Date, 1, Date, Vector{Date}} 2021-02-24 to 2021-12-01,
            percentEmbargo = 0.05),
    measure = cross_entropy(
            eps = 2.220446049250313e-16),
    weights = nothing,
    operation = MLJModelInterface.predict,
    range = NumericRange(
            field = :min_purity_increase,
            lower = 0.001,
            upper = 1.0,
         

In [9]:
labels = targetPositions
name = colnames(labels)[1] # labels name
data, labels = DataFrame(data), DataFrame(labels) # convert timearrays to dataframe
select!(data, Not(:timestamp)) # remove timestamp
select!(labels, Not(:timestamp)) # remove timestamp

sample = hcat(data, labels) # merge entire data 
for column ∈ names(sample)
    if eltype(sample[!, column]) == Float64 || eltype(sample[!, column]) == Int # check scitypes
        sample = coerce(sample, Symbol(column) => Continuous) # change scitypes
    else
        sample = coerce(sample, Symbol(column) => Multiclass) # change scitypes
    end    
end
labels, data = unpack(sample, ==(name), colname -> true) # unpack data


(CategoricalValue{String, UInt32}["1.0", "1.0", "1.0", "1.0", "1.0", "1.0", "1.0", "1.0", "1.0", "1.0"  …  "1.0", "1.0", "1.0", "1.0", "1.0", "1.0", "1.0", "1.0", "0.0", "0.0"], 281×3 DataFrame
│ Row │ Close   │ EMA 55  │ EMA 15  │
│     │ Float64 │ Float64 │ Float64 │
├─────┼─────────┼─────────┼─────────┤
│ 1   │ 49705.3 │ 39808.0 │ 49887.1 │
│ 2   │ 47093.9 │ 40068.2 │ 49537.9 │
│ 3   │ 46339.8 │ 40292.2 │ 49138.1 │
│ 4   │ 46188.5 │ 40502.8 │ 48769.4 │
│ 5   │ 45137.8 │ 40668.3 │ 48315.5 │
│ 6   │ 49631.2 │ 40988.4 │ 48479.9 │
│ 7   │ 48379.0 │ 41252.4 │ 48467.3 │
│ 8   │ 50538.2 │ 41584.0 │ 48726.2 │
│ 9   │ 48561.2 │ 41833.2 │ 48705.6 │
│ 10  │ 48927.3 │ 42086.6 │ 48733.3 │
⋮
│ 271 │ 58730.5 │ 58529.9 │ 61110.6 │
│ 272 │ 56289.3 │ 58449.8 │ 60507.9 │
│ 273 │ 57569.1 │ 58418.4 │ 60140.5 │
│ 274 │ 56280.4 │ 58342.0 │ 59658.0 │
│ 275 │ 57274.7 │ 58303.9 │ 59360.1 │
│ 276 │ 53569.8 │ 58134.8 │ 58636.3 │
│ 277 │ 54815.1 │ 58016.3 │ 58158.7 │
│ 278 │ 57248.5 │ 57988.8 │ 58044.9 │
│ 279 

In [16]:
mach = machine(self_tuning_tree, data, labels);
fit!(mach, verbosity=0)

Machine{ProbabilisticTunedModel{Grid,…}} @159 trained 1 time.
  args: 
    1:	Source @660 ⏎ `Table{AbstractVector{Continuous}}`
    2:	Source @247 ⏎ `AbstractVector{Multiclass{2}}`


In [17]:
fitted_params(mach).best_model

DecisionTreeClassifier(
    max_depth = -1,
    min_samples_leaf = 1,
    min_samples_split = 2,
    min_purity_increase = 1.0,
    n_subfeatures = 0,
    post_prune = false,
    merge_purity_threshold = 1.0,
    pdf_smoothing = 0.0,
    display_depth = 5,
    rng = Random._GLOBAL_RNG()) @074